In [10]:
import polars as pl
import pandas as pd
import prince
import numpy as np

import preprocessing_data as ppd
print(pl.__version__)

1.31.0


In [8]:
import os
print(os.getcwd())

D:\AI experiments\Customer_IT_Support-Ticket_Dataset


In [72]:
import spacy
from spacy.tokens import DocBin

# Loading & cleaning

In [38]:
#Data training
df=ppd.make_clean("data/aa_dataset-tickets-multi-lang-5-2-50-version.csv")
df_train = df.filter(pl.col("language") == "en")
train_corpus=df_train["body"]
train_corpus

No difference between after cleaning


body
str
"""Dear Customer Support Team,\n\…"
"""Dear Customer Support Team,\n\…"
"""Dear Customer Support Team,\n\…"
"""Dear Support Team,\n\nI hope t…"
"""Dear Customer Support,\n\nI ho…"
…
"""An unexpected billing discrepa…"
"""A data breach has occurred, wh…"
"""The data analytics tool experi…"


In [15]:
#Data test
df_test = ppd.make_clean("data/dataset-tickets-multi-lang3-4k.csv")
df_test=df_test.filter(pl.col("language")=="en")
test_corpus= df_test["body"]
test_corpus

Some elements have been dropped during cleaning


body
str
"""Dear IT Services Support Team,…"
"""Dear Customer Support, I am e…"
"""Dear IT Services Customer Supp…"
"""Dear Tech Online Store Support…"
"""Dear Customer Support, I hope…"
…
"""Hello Customer Support Team, …"
"""Dear Customer Support Team, W…"
"""Dear Customer Support Team, O…"


# Lowercasing

In [31]:
train_corpus=list(train_corpus)
train_corpus = [sentence.lower() for sentence in train_corpus]

In [32]:
test_corpus=list(test_corpus)
test_corpus = [sentence.lower() for sentence in test_corpus]

# Lemmatising

In [36]:
len("Apple is looking at buying U.K. startup for $1 billion")

54

# #_

In [66]:
labels = list(df_train["queue"].unique())

TRAIN_DATA = []
df_train = pd.DataFrame(df_train, columns=df_train.columns)
for _, row in df_train.iterrows():
    cats = {label: 1.0 if label == row["queue"] else 0.0 for label in labels}
    TRAIN_DATA.append((row["body"], {"cats": cats}))


In [67]:
labels = list(df_test["queue"].unique())

TEST_DATA = []
df_test = pd.DataFrame(df_test, columns=df_test.columns)
for _, row in df_test.iterrows():
    cats = {label: 1.0 if label == row["queue"] else 0.0 for label in labels}
    TEST_DATA.append((row["body"], {"cats": cats}))


In [70]:
def convert_to_spacy(data, output_file):
    nlp = spacy.blank("en")
    db = DocBin()
    for text, annotations in data:
        doc = nlp.make_doc(text)
        doc.cats = annotations["cats"]
        db.add(doc)
    db.to_disk(output_file)



In [74]:
print(os.getcwd())

D:\AI experiments\Customer_IT_Support-Ticket_Dataset


In [73]:
convert_to_spacy(TRAIN_DATA[:10000], "train.spacy")
convert_to_spacy(TRAIN_DATA[10000:], "valid.spacy")

In [75]:
convert_to_spacy(TEST_DATA, "test.spacy")